# Time plot Diagnostics

In [ ]:
# Use magic commands (%)

# When modifying libraries, no need to restart the kernel
%matplotlib inline  
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib

import src.copa.util_functions as u

In [ ]:
u.setPandasOptions()

In [ ]:
f1 = pd.read_csv("data/Copa-FSU-02172021_comma separated.csv") #, nrows=10000)

In [ ]:
#f2,_ = u.cleanupColumns(f1)

In [ ]:
#f2.to_csv("data/fsu_clean.csv", index=0)

In [ ]:
# Only 72 columns left. All DTMZ dates are integers
fsu = pd.read_csv("data/fsu_clean.csv") 

# Make sure it is reading from the Rosetta Stone data folder
#   x.csv only exiss in the Rosetta Stone data folder
#fsu = pd.read_csv("data/x.csv") 

In [ ]:
#fsu.to_csv("data/fsu_clean.csv", index=0)

In [ ]:
# Store dates as integers (nanoseconds)
fsu['IN_DTMZ'].head(3)

In [ ]:
# Make sure pd.to_datetime works
dt = pd.to_datetime(fsu['IN_DTMZ'])
dt.head(3)

In [ ]:
fsu = u.removeCancellations(fsu)
fsu = u.removeTurnbacks(fsu)
fsu = u.removeDiversions(fsu)

# Flights
* Every row is a different flight
* Flight by hour, day, week, month

## General Strategy
* Collect the columns of interest into a Dataframe
* Work with that dataframe
* Much faster since less data

In [ ]:
data = fsu['FLT_NUM']

In [ ]:
fsu

In [ ]:
data.hist(bins=50)

# Observation: there are two sets of flights
* We learned from Copa that we should only consider FLT_NUM between 0 and 999.
* As seen below, there are 4051 flights not CM. We remove them. 

In [ ]:
(fsu.AC_OWNER_CARRIER_CD != 'CM').sum()

In [ ]:
fsu = u.filterFlights(fsu)

In [ ]:
fsu['FLT_NUM'].hist(bins=30)

* We observe that the higher the flight number, the lower the number of flights. 
* This suggests that lower flights numbers flight more often.

## Number of flights per day
* Aggregate by day and compute the number of flights per hour
* We must extract the hour from the database 
* We then aggregate the number of flights by hour

In [ ]:
tmz = u.series_to_time_components(fsu['SCH_DEP_TMZ'])
tmz.head()

* Compute the average number of flights (departures and arrivals) per hour

In [ ]:
nb_days = fsu['SCH_DEP_DTZ'].nunique()
fsu1 = fsu.copy()  # prevent slicing program (I do not fully understand)
print("Nb unique days: ", nb_days)
# nb flights per day
fsu1['h'] = tmz['h']
fsu = fsu1.copy()
# Average number of daily flights each hour
fsu['per_hour'] = fsu.groupby('h')['FLT_NUM'].transform('size')  / nb_days

## Observations
* It is a good idea to remove Warnings. 
* transform() is used with groupby to create a column of aggregated data over subgroups
    * sum, mean, std, etc

In [ ]:
# Average number of flights per hour
fsu['per_hour'].head(5), fsu.shape

* Compute the average number of flights per hour, averaged over all days

In [ ]:
data = fsu[['per_hour','h']]

In [ ]:
data;

In [ ]:
# Choose bins carefully, choose what to plot carefully
data.hist(bins=30)

# Plot Mean values as a function of hour

# Plot as much data as possible 
* Balance between plot complexity and plot insights
* One plot or multiple plots?
    * Case by case basis

In [ ]:
data.columns

In [ ]:
data.boxplot(column='per_hour', by='h', figsize=(10,4))

## Observation
* The box plot is a poor choice since each group has the same per-hour daily average

In [ ]:
data.groupby('h')['per_hour'].mean().plot()

## Observation
* The line plot is useful to connect points over a coordinate direction (time or x or y)
* There are more flights around 1 am, and between 11h an 22h
* HOWEVER: the times are Zulu, so subract 5 hours to get PTY time
* Thus, there are more flights around 8 pm, and between 6am and 5 pm. 
    * Makes sense

In [ ]:
# Same plot as before, but as a bar chart. 
# The content is identical to the lineplot but looks nicer
data.groupby('h')['per_hour'].mean().plot(x='h',y='per_hour',kind='bar')

# Are the flights per hour correct? 
* The times are DTMZ, so I must subtract 5 hours
* mod 24, and sort the hours to get a range [0,24]
* So we replot

In [ ]:
hours = data.groupby('h')['per_hour'].mean().reset_index() #to_frame('per_hour')
hours['h'] = ((hours['h'] - 5 + 24) % 24)   # Convert to panama time
hours.sort_values('h', inplace=True)
hours;
#“hours.plot(x='h',y='per_hour',kind='bar')

In [ ]:
hours.plot(x='h',y='per_hour',kind='bar') 

## Observations
* The plots suggest a peak bretween 7-9 am, 3 pm, 18 pm, and 21 pm.
* Let us separate arrivals and departures
* Create a column with 'dep' and 'arr' symbols
* For this, return to fsu dataframe
* Later, during the course of debugging, I will create to transform time from Zulu to Local

In [ ]:
# Make sure that ORIG_CD and DEST_CD are consistent with OD
# In other words, make sure that OD = ORIG_CD + DEST_CD
# This is not always the case. But we have removed diversions, cancellations, etc
fsu[(fsu['ORIG_CD']=='PTY') & (fsu['OD'].str[0:3] != 'PTY')]

## Observations
* In my original exploration, there were 9 flights in the next cell. 
* They are probably cancellations, diversions, turnbacks, etc. 
    * We do not worry about them. We removed them in a cleanup function.

In [ ]:
# Not clear why there are still 4 rows left
fsu[(fsu['DEST_CD']=='PTY') & (fsu['OD'].str[3:6] != 'PTY')].shape

In [ ]:
nodiversion = (fsu['OD'] != (fsu['ORIG_CD']+fsu['DEST_CD']))
fsu[nodiversion].shape, fsu.shape

## All problems stated here have been fixed in method cleanColumns()
* Notice that these are all diversions. 
* However, I thought I removed diversions. How to identify these
    * 1) by the FLT_TYPE_NAME containing "diversion"
    * 2) ORIGIN_PLANNED != ORIG_CD  (takes care of 3 out of 4)
    * 3) ORIG_CD == DEST_CD (this is a turnback, but they have been removed)
* Remove flights where 'ORIG_CD' + 'DEST_CD' != 'OD'
    * This takes care of all diversions

In [ ]:
# How is it possible to have OD=PTYPTY and it is classified as a diversion?
fsu1 = fsu[(fsu['DEST_CD']=='PTY') & (fsu['OD'].str[3:6] != 'PTY')]
fsu1.to_csv("Strange_diversion.csv", index=0)
fsu1

In [ ]:
# Make sure that ORIG_CD and DEST_CD are consistent with OD
# all is now consistent
fsu[(fsu['ORIG_CD']=='PTY') & (fsu['OD'].str[0:3] != 'PTY')]

In [ ]:
fsu['dep-arr'] = 'dep'
fsu.loc[fsu['OD'].str[3:6] == 'PTY', 'dep-arr'] = 'arr'

In [ ]:
# Without the copy, there is a slicing warning, and wy take chances?
data.loc[:,'dep-arr'] = fsu.copy().loc[:,'dep-arr']
data.columns

In [ ]:
hours = data.groupby(['h','dep-arr'])['per_hour'].mean().reset_index() #to_frame('per_hour')
hours['h'] = ((hours['h'] - 5 + 24) % 24)   # Convert to panama time
hours.sort_values('h', inplace=True)
#“hours.plot(x='h',y='per_hour',kind='bar')

In [ ]:
# Hours are now in PTY time
# Departures start arund 7 am, with peaks at 9 am, 15 pm, 18 pm, 21 pm
# Not many arri vals after 6 pm

plt.figure(figsize=(10,5))
ax = sns.barplot(data=hours,x='h',y='per_hour',hue='dep-arr')
ax.set_xlim(0,23)

## Next issue: arrivals is equal to departures on many hours. 
* Is this true? Let us plot a single day
* We are creating many similar plots: Time for a user-deifned function!

In [ ]:
# Strictly speaking, I should do the scan using PTY time.
# So the results are only approximatley correct
fsu_oneday_dep = fsu[(fsu['SCH_DEP_DTZ'] == '2020-02-13') & (fsu['ORIG_CD'] == 'PTY')].copy()
fsu_oneday_dep.loc[:, 'dep-arr'] = 'dep'
fsu_oneday_arr = fsu[(fsu['SCH_ARR_DTZ'] == '2020-02-13') & (fsu['DEST_CD'] == 'PTY')].copy()
fsu_oneday_arr.loc[:, 'dep-arr'] = 'arr'

In [ ]:
fsu['IN_DTMZ'].dtype

In [ ]:
# Careful. Some words cannot be variables. E.g. "in" is not allowed. So use "inz"
# The following lines are in preparation of using actual time of departure and arrival
inz = pd.to_datetime(fsu['IN_DTMZ']).dt.date
outz = pd.to_datetime(fsu['OUT_DTMZ']).dt.date

fsu_oneday_in  = fsu[(inz == '2020-02-13') & (fsu['DEST_CD'] == 'PTY')]
fsu_oneday_out = fsu[(inz == '2020-02-13') & (fsu['ORIG_CD'] == 'PTY')]

In [ ]:
def per_hour(fsu_dep1, fsu_arr1, title=""):
    """
    Plot numbrer of scheduled arrivals and departures per hour
    """
    fsu_dep = fsu_dep1.copy()
    fsu_arr = fsu_arr1.copy()
    
    nb_days = fsu_dep['SCH_DEP_DTZ'].nunique()
    tmz = u.series_to_time_components(fsu_dep['SCH_DEP_TMZ'])
    fsu_dep['h'] = tmz['h'].copy()
    fsu_dep['mi'] = tmz['mi'].copy()
    fsu_dep['per_hour'] = fsu_dep.groupby('h')['FLT_NUM'].transform('size')  / nb_days
    fsu_dep['dep-arr'] = 'dep'
    tot_num_dep = fsu_dep['per_hour'].sum()
            
    nb_days = fsu_arr['SCH_DEP_DTZ'].nunique()
    tmz = u.series_to_time_components(fsu_arr['SCH_ARR_TMZ'])
    fsu_arr['h'] = tmz['h'].copy()
    fsu_arr['mi'] = tmz['mi'].copy()
    fsu_arr['per_hour'] = fsu_arr.groupby('h')['FLT_NUM'].transform('size')  / nb_days
    fsu_arr['dep-arr'] = 'arr'
 
    fsu = pd.concat([fsu_dep, fsu_arr], axis=0)
    
    data = fsu[['per_hour','h','dep-arr']]
    hours = data.groupby(['h','dep-arr'])['per_hour'].mean().reset_index() #to_frame('per_hour')
    # -5 to convert from Zulu to Panama time
    hours['h'] = ((hours['h'] - 5 + 24) % 24)   # Convert to panama time
    hours.sort_values('h', inplace=True)
    
    deparr = hours.groupby('dep-arr').sum()
    print(deparr)
    print(f"total flights (dep+arr): {hours['per_hour'].sum()}")

    plt.figure(figsize=(10,5))
    plt.title(title, fontsize=24)
    ax = sns.barplot(data=hours,x='h',y='per_hour',hue='dep-arr')
    ax.set_xlim(0,23)
    labels = ax.get_xticklabels()
    ax.set_xticklabels(labels, fontsize=8);  # supress output
    plt.show()
    return 

In [ ]:
def per_hour_in_out(fsu_dep1, fsu_arr1, title=""):
    """
    Plot Number of flights actually departing and arriving per hour
    """
    fsu_dep = fsu_dep1.copy()
    fsu_arr = fsu_arr1.copy()
    nb_days = fsu_dep['SCH_DEP_DTZ'].nunique()
    print("nb_days= ", nb_days)
    
    tmz1 = u.series_to_time_components(pd.to_datetime(fsu_dep['OUT_DTMZ']))  ### Something wrong WRONG WRONG WRONG
    
    fsu_dep['h'] = tmz1['h'] 
    fsu_dep['mi'] = tmz1['mi']
    fsu_dep['per_hour'] = fsu_dep.groupby('h')['FLT_NUM'].transform('size')  / nb_days
    fsu_dep['dep-arr'] = 'dep'
    #tot_num_dep = fsu_dep['per_hour'].sum()
            
    nb_days = fsu_arr['SCH_DEP_DTZ'].nunique()
    tmz = u.series_to_time_components(pd.to_datetime(fsu_arr['IN_DTMZ']))
    fsu_arr['h'] = tmz['h']
    fsu_arr['mi'] = tmz['mi']
    fsu_arr['per_hour'] = fsu_arr.groupby('h')['FLT_NUM'].transform('size')  / nb_days
    fsu_arr['dep-arr'] = 'arr'
    #tot_num_arr = fsu_arr['per_hour'].sum()
    
    #print(f"tot num dep/arr: {tot_num_dep}/{tot_num_arr}")
    
    fsu = pd.concat([fsu_dep, fsu_arr], axis=0)
    
    data = fsu[['per_hour','h','dep-arr']]
    hours = data.groupby(['h','dep-arr'])['per_hour'].mean().reset_index() #to_frame('per_hour')
# -5 to convert from Zulu to Panama time
    hours['h'] = ((hours['h'] - 5 + 24) % 24)   # Convert to panama time from Zulu time
    hours.sort_values('h', inplace=True)
    
    deparr = hours.groupby('dep-arr').sum()
    print(deparr)
    
    print(f"total flights (dep+arr): {hours['per_hour'].sum()}")
    
    plt.figure(figsize=(10,5))
    plt.title(title, fontsize=24)
    ax = sns.barplot(data=hours,x='h',y='per_hour',hue='dep-arr')
    ax.set_xlim(0,23)
    labels = ax.get_xticklabels()
    ax.set_xticklabels(labels, fontsize=8);  # supress output
    plt.show()
    return

In [ ]:
per_hour(fsu_oneday_dep, fsu_oneday_arr, "Scheduled departure/arrival times")
cols = ['SCH_DEP_DTZ','FLT_NUM','IN_DTMZ','OUT_DTMZ']
fsu_oneday_dep[cols].columns
per_hour_in_out(fsu_oneday_dep[cols], fsu_oneday_arr[cols], title="Actual departure/arrival times")

## Observations
* How can the total flights (dep+arr) have a fractional component? There must be an error. 
    * Why? I am averging over a single day, so fractions can not possibly occur. 
* The total number of scheduled departures+arrivals in one day is 241.5 (avg of 14 flights per hour throughout the day)
* The total number of actual departures an arrivals+departures in one day is also 241.5 . That is consistent.

* This plot looks much better. 
* Arrivals are no longer equal to departures
* Several errors were fixed
* However, it seems like there are far more departures than arrivals. 
    * We need to check this  (THERE WERE IN FACT ERRORS)
* There are very few deparatures at 1 and 2 pm. Very few at 10 and 11 pm
* There a low number of flights around 4 and 5 pm (either arrival or departure)

In [ ]:
fsu_oneday_dep.shape, fsu_oneday_arr.shape

Clearly, the number of departures should be approximately equal to the number of arrivals. Why? Because PTY is a hub!
* Next step: break down time in 15 in increments
* Eventually, break down time in 5 min incremements
    * We will need to add time slots for which there are no flights

In [ ]:
def per_15min(fsu_dep1, fsu_arr1, title=""):
    fsu_dep = fsu_dep1.copy()
    fsu_arr = fsu_arr1.copy()
    #fsu.loc[:,'dep-arr'] = 'dep'
    #fsu.loc[fsu['OD'].str[3:6] == 'PTY', 'dep-arr'] = 'arr'
    #data.loc[:,'dep-arr'] = fsu.loc[:,'dep-arr']
    nb_days = fsu_dep['SCH_DEP_DTZ'].nunique()
    nb_days = 1
    
    tmz = u.series_to_time_components(fsu_dep['SCH_DEP_TMZ'])
    #fsu_dep['h'] = tmz['h'].copy()
    fsu_dep['mi'] = tmz['mi'].copy()
    fsu_dep['15min'] = (tmz['h']*60+tmz['mi']) // 15
    fsu_dep['per_15min'] = fsu_dep.groupby('15min')['FLT_NUM'].transform('size')  / nb_days
    fsu_dep['dep-arr'] = 'dep'
    tot_num_dep = fsu_dep['per_15min'].sum()
            
    nb_days = fsu_arr['SCH_DEP_DTZ'].nunique()
    nb_days = 1
    
    tmz = u.series_to_time_components(fsu_arr['SCH_ARR_TMZ'])
 
    #fsu_arr['h'] = tmz['h'].copy()
    fsu_arr['mi'] = tmz['mi'].copy()
    fsu_arr['15min'] = (tmz['h']*60+tmz['mi']) // 15
    
    fsu_arr['per_hour'] = fsu_arr.groupby('h')['FLT_NUM'].transform('size')  / nb_days
    fsu_arr['per_15min'] = fsu_arr.groupby('15min')['FLT_NUM'].transform('size')  / nb_days
    fsu_arr['dep-arr'] = 'arr'
    tot_num_arr = fsu_arr['per_15min'].sum()
    
    print(f"tot num dep/arr: {tot_num_dep}/{tot_num_arr}")
    
    fsu = pd.concat([fsu_dep, fsu_arr], axis=0)
    
    data = fsu[['per_15min','15min','dep-arr']]
    hours = data.groupby(['15min','dep-arr'])['per_15min'].mean().reset_index() #to_frame('per_hour')
    # -5 to convert from Zulu to Panama time
    #hours['h'] = ((hours['h'] - 5 + 24) % 24)   # Convert to panama time
    hours['15min'] = ((hours['15min'] - 5*4 + 24*4) % (24*4)) / 4  # Convert to panama time
    hours.sort_values('15min', inplace=True)
    
    deparr = hours.groupby('dep-arr').sum()
    print(deparr)
    
    print(f"total flights (dep+arr): {hours['per_15min'].sum()}")
    
    plt.figure(figsize=(18,5))
    plt.title(title, fontsize=24)
    ax = sns.barplot(data=hours,x='15min',y='per_15min',hue='dep-arr')
    ax.set_xlim(0,23*4)
    labels = ax.get_xticklabels()
    ax.set_xticklabels(labels, fontsize=8);  # supress output
    plt.xticks(rotation=45)
    plt.show()
    return 

In [ ]:
def per_15min_in_out(fsu_dep1, fsu_arr1, title=""):
    fsu_dep = fsu_dep1.copy()
    fsu_arr = fsu_arr1.copy()
    #fsu.loc[:,'dep-arr'] = 'dep'
    #fsu.loc[fsu['OD'].str[3:6] == 'PTY', 'dep-arr'] = 'arr'
    #data.loc[:,'dep-arr'] = fsu.loc[:,'dep-arr']
    nb_days = fsu_dep['SCH_DEP_DTZ'].nunique()
    #nb_days = 1
 
    #tmz = u.series_to_time_components(fsu_dep['SCH_DEP_TMZ'])
    tmz = u.series_to_time_components(pd.to_datetime(fsu_dep['OUT_DTMZ']))  ### Something wrong WRONG WRONG WRONG
    #fsu_dep['h'] = tmz['h'].copy()
    fsu_dep['mi'] = tmz['mi'].copy()
    fsu_dep['15min'] = (tmz['h']*60+tmz['mi']) // 15
    fsu_dep['per_15min'] = fsu_dep.groupby('15min')['FLT_NUM'].transform('size')  / nb_days
    fsu_dep['dep-arr'] = 'dep'
    tot_num_dep = fsu_dep['per_15min'].sum()
            
    nb_days = fsu_arr['SCH_DEP_DTZ'].nunique()
    #nb_days = 1
    
    tmz = u.series_to_time_components(pd.to_datetime(fsu_arr['IN_DTMZ']))  ### Something wrong WRONG WRONG WRONG
    #tmz = u.series_to_time_components(fsu_arr['SCH_ARR_TMZ'])
    #fsu_arr['h'] = tmz['h'].copy()
    fsu_arr['mi'] = tmz['mi'].copy()
    fsu_arr['15min'] = (tmz['h']*60+tmz['mi']) // 15
    
    fsu_arr['per_hour'] = fsu_arr.groupby('h')['FLT_NUM'].transform('size')  / nb_days
    fsu_arr['per_15min'] = fsu_arr.groupby('15min')['FLT_NUM'].transform('size')  / nb_days
    fsu_arr['dep-arr'] = 'arr'
    tot_num_arr = fsu_arr['per_15min'].sum()
    
    print(f"tot num dep/arr: {tot_num_dep}/{tot_num_arr}")
    
    fsu = pd.concat([fsu_dep, fsu_arr], axis=0)
    
    data = fsu[['per_15min','15min','dep-arr']]
    hours = data.groupby(['15min','dep-arr'])['per_15min'].mean().reset_index() #to_frame('per_hour')
    # -5 to convert from Zulu to Panama time
    #hours['h'] = ((hours['h'] - 5 + 24) % 24)   # Convert to panama time
    hours['15min'] = ((hours['15min'] - 5*4 + 24*4) % (24*4)) / 4  # Convert to panama time
    hours.sort_values('15min', inplace=True)
    
    deparr = hours.groupby('dep-arr').sum()
    print(deparr)
    
    print(f"total flights (dep+arr): {hours['per_15min'].sum()}")

    plt.figure(figsize=(18,5))
    plt.title(title, fontsize=24)
    ax = sns.barplot(data=hours,x='15min',y='per_15min',hue='dep-arr')
    ax.set_xlim(0,23*4)
    labels = ax.get_xticklabels()
    ax.set_xticklabels(labels, fontsize=8);  # supress output
    plt.xticks(rotation=45)
    plt.show()
    return 

In [ ]:
per_15min(fsu_oneday_dep, fsu_oneday_arr, title="Scheduled departure/arrivals, 15 min bins")
per_15min_in_out(fsu_oneday_dep, fsu_oneday_arr,"Actual departure/arrivals, 15 min bins")

## Observations on the departures/arrivals plots
* Here are 15 min bins. Above was 1 hour bins. 
* You notice that the actual departures and arrivals have overlaps at certain times of the day. 
* I wonder if the TAXI_IN and TAXI_OUT times are also longer than usual on the flights on that particular day during these congestion days. 
* It does look like COPA tries to avoid departures and arrivals during the same time period in order to improve the traffic flow on the runways perhaps? I assume that if all planes go in a single direction (arrivals or departures) that flow management is simpler? For example, you can have more planes leave at the same time. 
* I like these plots. 
* On reflection, I must check whether the sum of all flights on both plots is the same. It does not appear to be the case. It "looks" like there are less arrivals on the bottom plot. I now have to check this. 

## Number of flights (departure + arrivals) is now computed.
* I find 241.5 actual departures/arrivals (in 15min increments)
* I find 321 scheduled departures/arrivals (in 15 min increments)
* I conclude that since the total number of flights is 241.5 when using hourly bins, that something must be wrong with the schdeuled flights in 15min increments. 
* Although it is possible that the total number of flights might be different when comparing scheduled and actual flights (e.g., a flight scheduled to land late at night might actually land the next morning, and a flight might actually arrive early in the morning that was not scheduled for that morning), these should be low in number. However, it should be remembered that delays are highest at the end of the day. 

## Found a first bug
* I was adding up the wrong data
* I still have not figured out the fractional value point issue, but it has to do with using the 'per_hour' column, instead of the 'hour' column for aggregation

### At this stage, I should consolidate the four functions above into a single function to minimize propagation of errors
* We will start with scheduled time. My expectations are as followed (from a diagnostic point of view): 
    * the total number of arrivals and departures should be integer values
    * the total number of arrivals and departures should be the same whether binning per hour or per 15 min
* I created the function `per_interval`,    took almost a full day to create and debug

## Features of per_interval function in util_functions.py
* can specify a range of dates
* can specify Zulu or PTY time zone 
* can specify either SCHED or ACTUAL time
* can specify to show only flights with delays in a specified range

In [ ]:
f, ax = plt.subplots(figsize=(10,3))
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', date='2020-02-13', bin_size=60, title="", ax=ax)
f, ax = plt.subplots(figsize=(10,3))
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', date='2020-02-13', bin_size=15, title="", ax=ax)
f, ax = plt.subplots(figsize=(10,3))
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', date='2020-02-13', bin_size=5, title="", ax=ax)

In [ ]:
f, axes = plt.subplots(3,2, figsize=(20,8))
pd.options.mode.chained_assignment = None  # 'warn', 'raise'
plt.suptitle("Number of Departing and Arriving flights per \nunit time, 2020-02-13", fontsize=24)
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', bin_size=60, title="SCHED, 60min bins", date='2020-02-13', ax=axes[0,0])
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', bin_size=15, title="SCHED, 15min bins", date='2020-02-13', ax=axes[1,0])
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', bin_size=5, title="SCHED, 5min bins", date='2020-02-13', ax=axes[2,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=60, title="ACTUAL, 60min bins", date='2020-02-13', ax=axes[0,1])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=15, title="ACTUAL, 15min bins", date='2020-02-13', ax=axes[1,1])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=5, title="ACTUAL, 5min bins", date='2020-02-13', ax=axes[2,1])
pd.options.mode.chained_assignment = 'raise'  # 'warn', 'raise'
plt.tight_layout()

In [ ]:
f, axes = plt.subplots(3,2, figsize=(20,8))
pd.options.mode.chained_assignment = None  # 'warn', 'raise'
plt.suptitle("Number of Departing and Arriving flights per \nunit time, 2020-02-13", fontsize=24)
dates=('2019-10-01','2020-03-01')
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', bin_size=60, title="SCHED, 60min bins", ax=axes[0,0], daterange=dates)
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', bin_size=15, title="SCHED, 15min bins", ax=axes[1,0], daterange=dates)
u.per_interval(fsu, desired_timezone='PTY', kind='SCHED', bin_size=5, title="SCHED, 5min bins", ax=axes[2,0], daterange=dates)
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=60, title="ACTUAL, 60min bins", ax=axes[0,1], daterange=dates)
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=15, title="ACTUAL, 15min bins", ax=axes[1,1], daterange=dates)
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=5, title="ACTUAL, 5min bins", ax=axes[2,1], daterange=dates)
pd.options.mode.chained_assignment = 'raise'  # 'warn', 'raise'
plt.tight_layout()

## Observations
* Scheduled departure/arrivals:  160/162  (makes sense)
* Actual departure/arrivals:  159/162  (makes sense)
* I have no idea what the previous errors were, but the moral of the story: 
    * Create more general routines that apply to more cases instead of many specialized routines
    * Stated differently: try not to duplicate code. 
    * Increase reusability

* Discrepencies between scheduled and actual can be seen around 7:30 am departures, and around 8 pm. 
* It would be useful to either :
    * plot only the flights with over 15 positive delays (on arrival or departure)
    * plot the difference in flights per hour between SCHED and ACTUAL
    * correlate these plots with bank activity (perhaps via color?). 

In [ ]:
dates = ('2019-10-01', '2020-03-01')
f, axes = plt.subplots(2,2,figsize=(20,7))
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=5, daterange=dates, 
              title="All flights, 5min bins", ax=axes[0,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=15, 
               max_delay=45, bin_size=5, daterange=dates, title="Delays in [15,45], 5min bins", ax=axes[0,1])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=15,  bin_size=5, 
               daterange=dates, title="Delays in [15,inf], 5min bins", ax=axes[1,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=45,  bin_size=5, 
               daterange=dates, title="Delays in [45,inf], 5min bins", ax=axes[1,1])
plt.tight_layout()

In [ ]:
dates = ('2019-12-20', '2020-01-10')
f, axes = plt.subplots(2,2,figsize=(20,7))
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=5, daterange=dates, 
              title="All flights, 5min bins", ax=axes[0,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=15, 
               max_delay=45, bin_size=5, daterange=dates, title="Delays in [15,45], 5min bins", ax=axes[0,1])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=15,  bin_size=5, 
               daterange=dates, title="Delays in [15,inf], 5min bins", ax=axes[1,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=45,  bin_size=5, 
               daterange=dates, title="Delays in [45,inf], 5min bins", ax=axes[1,1])
plt.tight_layout()

In [ ]:
dates = ('2019-12-20', '2019-12-20')
f, axes = plt.subplots(2,2,figsize=(20,7))
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=5, daterange=dates, 
              title="All flights, 5min bins", ax=axes[0,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=15, 
               max_delay=45, bin_size=5, daterange=dates, title="Delays in [15,45], 5min bins", ax=axes[0,1])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=15,  bin_size=5, 
               daterange=dates, title="Delays in [15,inf], 5min bins", ax=axes[1,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=45,  bin_size=5, 
               daterange=dates, title="Delays in [45,inf], 5min bins", ax=axes[1,1])
plt.tight_layout()

In [ ]:
dates = ('2020-01-10', '2020-01-10')
f, axes = plt.subplots(2,2,figsize=(20,7))
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', bin_size=5, daterange=dates, 
              title="All flights, 5min bins", ax=axes[0,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=15, 
               max_delay=45, bin_size=5, daterange=dates, title="Delays in [15,45], 5min bins", ax=axes[0,1])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=15,  bin_size=5, 
               daterange=dates, title="Delays in [15,inf], 5min bins", ax=axes[1,0])
u.per_interval(fsu, desired_timezone='PTY', kind='ACTUAL', min_delay=45,  bin_size=5, 
               daterange=dates, title="Delays in [45,inf], 5min bins", ax=axes[1,1])
plt.tight_layout()

## Observations
* The four plots: 
    * top-left: all actual departures and arrival times: nb flights per hour
    * top-right: delays 45 min and above
    * bottom-left: delays 15 min and above
    * bottom-right: delays 45 min and above
* Beware: the scale of the vertical axes changes. 
* Observe in the bottom right: that te delays are higher at the end of day. 
* However: the delays between 15 and 45 min are highest between 2 and 3 pm. 
* You should know that these are not two plots (arr and dep) superimposed on top of each other. 
* Rather, at each bin, there are two bars (side by side): one for arrivals and one for departures. 
* This might be important to know for proper interpretation; this is more visible on the hourly and 15 min charts. 
* Long time frames provides prior probability distributions. 
* However, it is important to know how these results change as the time-interval captured is decreased, since ultimately, Copa is interested on making decisions for individual flights, taking into accounts other flights within say, a three hour window on the network immediately connected to the plane for which a decision is necessary.

# I have reached this point, 2021-04-25,11.26pm
# I have reached this point, 2021-04-25,11.26pm
# I have reached this point
# I have reached this point

## Observations
* I am not convinced that there only 50 flights per day. There should be 350 flights per day.

## Observations
* The thinner lines are due to "aliasing"
* It means that the bins are too small, or the plot not wide enough
* Why are delays clustered around 2 pm, mostly if not all on arrival? 
* Why is there an arrival delay cluster around 6 am? These must come from the previous evening. 
    * Therefore, plot delays greater than 45 min and see if they appear in the morning arrivals. 

# Something is still wrong. Flights disappear when I change binning

In [ ]:
f, axes = plt.subplots(1,1, figsize=(20,3))
u.per_interval(fsu, kind='ACTUAL', desired_timezone='PTY',min_delay=-100, bin_size=5, title="Delay in [15,inf], 5min bins, [2019-12-19 to 2020-01-15]", daterange=('2019-12-19','2020-01-15'), ax=axes)

## Observations
* The delays on arrival early in the morning are mostly less than 45 min. 
* Conclusion they come from international flights? 
* It is possible that stacked barcharts might be better. But they are hard to compare quantitatively with each other. 
    * The issue is that the range corresponding to each bar is difficult to see when doing detailed analysis
* The list above has 16 entries. However, there are 19 flights on the chart above
    * Is this an error? 
    * Note that I am using DTMZ times. PTY is 5 hours ahead. 12:00 DTZ is 5:00 DTL
    * I search the flights to include in the chart based on DTMZ. But I plot the results in PTY time. 
    * A flight arriving at 9 pm Zulu on 2020-01-15 , would actually be arriving at 2 am PTY time the next day and would be excluded from the plot
    * On the other hand, a flight arriving at 9 pm Zulu on 2019-12-18 (outside the date range) would appear on this plot. 
    * To fix the problem requires transforming from Zulu to DTML/PTY PRIOR to performing the record search (WE WON'T DO THIS HERE)
    * The fix is easy since I store datetimes in nanoseconds: 5 hours = 3600*1e9 = 36e11 nanoseconds. Just add that to the integer and convert back
* Add some more arguments to the function: timezone=['Zulu','PTY'] with default 'Zulu'

In [ ]:
pwd()

In [ ]:
f, axes = plt.subplots(1,1, figsize=(20,3))
u.per_interval(fsu, kind='SCHED', min_delay=None, bin_size=60, desired_timezone='PTY', title="Delay in [5,inf], [2019-12-19 to 2020-01-15]", daterange=('2020-02-13','2020-02-13'), ax=axes)

In [ ]:
f, axes = plt.subplots(1,1, figsize=(20,3))
u.per_interval(fsu, kind='ACTUAL', min_delay=None, bin_size=1, desired_timezone='PTY', title="Actual flights [-Inf,inf], [2019-12-19 to 2020-01-15]", daterange=('2020-02-13','2020-02-13'), ax=axes)